In [1]:
# Import packages
import os
import glob
import shutil
import plotnine

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers

from plotnine import ggplot
from plotnine import aes 
from plotnine import geom_line
from plotnine import labs

In [2]:
# Reproducibility
np.random.seed(478)
tf.random.set_seed(478)

In [3]:
# Read in data from .csv files
train = pd.read_csv('./Data/train.csv')
test = pd.read_csv('./Data/test.csv')

# Convert Pandas dataframes to valid Numpy arrays
def convert(X):
    data = np.empty((X.shape[0], 28, 28, 1))
    for i in range(X.shape[0]):
        img = np.array(X.iloc[i, :]).reshape((28, 28, 1))
        data[i] = img
    return(np.array(data))

# Define training set
X_train = convert(train.drop('label', axis = 1)) / 255
y_train = train['label']

# Define testing set
X_test = convert(test) / 255

In [4]:
model = keras.Sequential([
    layers.BatchNormalization(),
    layers.DepthwiseConv2D(kernel_size = 3, activation = 'relu', depth_multiplier = 2, input_shape = [28, 28, 1]),
    layers.MaxPool2D(),
    layers.Dropout(rate = 0.3),
    
    layers.BatchNormalization(),
    layers.DepthwiseConv2D(kernel_size = 3, activation = 'relu', depth_multiplier = 2),
    layers.MaxPool2D(),
    layers.Dropout(rate = 0.3),
    
    layers.Flatten(),
    layers.Dense(units = 10, activation = tf.nn.softmax)
])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=10, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# Compile model
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

# Train model with 75/25 training/validation split
trained = model.fit(
    X_train,
    y_train,
    epochs = 20,
    validation_split = .25,
    callbacks = [early_stopping]#,
    #verbose = 0
)

pd.DataFrame(trained.history)

Epoch 1/20
985/985 [==============================] - 27s 26ms/step - loss: 1.3850 - accuracy: 0.5216 - val_loss: 0.5782 - val_accuracy: 0.8719
Epoch 2/20
985/985 [==============================] - 28s 28ms/step - loss: 0.7173 - accuracy: 0.7642 - val_loss: 0.3910 - val_accuracy: 0.9009
Epoch 3/20
985/985 [==============================] - 26s 27ms/step - loss: 0.6235 - accuracy: 0.7968 - val_loss: 0.3615 - val_accuracy: 0.9114
Epoch 4/20
985/985 [==============================] - 27s 27ms/step - loss: 0.5910 - accuracy: 0.8075 - val_loss: 0.4993 - val_accuracy: 0.8980
Epoch 5/20
985/985 [==============================] - 26s 27ms/step - loss: 0.5818 - accuracy: 0.8095 - val_loss: 0.3868 - val_accuracy: 0.9118
Epoch 6/20
985/985 [==============================] - 26s 26ms/step - loss: 0.5708 - accuracy: 0.8149 - val_loss: 0.4465 - val_accuracy: 0.8919
Epoch 7/20
985/985 [==============================] - 27s 27ms/step - loss: 0.5610 - accuracy: 0.8177 - val_loss: 0.3733 - val_accuracy:

In [ ]:
# Generate competition preedictions based on test set
pred = pd.DataFrame(model.predict(X_test)).idxmax(axis = 1)
pred.index += 1
pred.head()
#pred.to_csv("C:\\Users\\Logan\\Desktop\\CSU\\Senior 2\\DSCI 478\\predictions.csv")